# U-Net for Crop Classification 

## Introduction

This notebook investigates the performance of a U-Net model in crop classification using the Zueri Crop dataset. Various configurations of the U-Net model will be explored. The goal is to analyze how the hyperparameters impact the model's accuracy and generalization on crop classification tasks. The notebook utilizes the `DeepModel_Trainer` class for data loading and model training, `UNet_small` and `UNet_Dropout` as modell architecture  and evaluation metrics are logged using WandB for comprehensive analysis.

The inputs for the U-Net are upsampled from 24 x 24 pixels to 80 x 80 pixels with the method of bicubic upsampling. The encoder part of the network consists of four downsampling blocks, each composed of two convolutional layers with ReLU activation followed by max-pooling, progressively reducing the spatial dimensions of the input. The encoder also includes dropout layers with a specified dropout rate applied to the fourth downsampling block. The decoder part consists of four upsampling blocks, each involving a transpose convolution operation and two convolutional layers with ReLU activation. Skip connections concatenate feature maps from the encoder to the decoder, helping preserve spatial information. The final output is produced by a 1x1 convolutional layer to generate segmentation masks. 

## Imports of libraries

In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from random import randrange
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import wandb
from sklearn.model_selection import KFold
from IPython.display import IFrame
from src.dataset import Dataset
from src.modelling import DeepModel_Trainer
from models.UNet_small import UNet_small
from models.UNet_Dropout import UNet
from torcheval.metrics.functional import multiclass_f1_score
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
wandb.login(key = '***')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: daniela-herzig (dlbs_crop). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /users/dherzig/.netrc


True

# Overfitting

For overfitting the Trainer_loader consists only of one batch. The model should learn the batch fast and should decrease the train loss substantially and the test loss should decrease because of the overftting and the missing generalization of the single batch for training.

In [3]:
Unet_small=UNet_small()

In [4]:
overfit = DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_small)
overfit.train_model('Overfitting', 'Unet_small', 'all', num_epochs=300, test_model=True, lr=5e-4, batch_size=4)

# First run unregularized

Because of the experience of the Challenge X, we runned a first model with an augmentation rate of 0.66 and no regularization, this were the default values from the ETHZ Paper described. It is a way of a baseline for the U-Net to get a feeling for the next regularization techniques.

In [ ]:
first_run = DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_small)
first_run.train_model('Run-unregularized', 'Unet_small', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4)

# Regularize
## Dropout
Different dropout rates are implemented in the fourth downsampling block and in the long-format downsampling block in the middle of the U-Net network.

In [ ]:
Unet_dropout = UNet(rate=0.5)

In [ ]:
Dropout_05 = DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_dropout)
Dropout_05.train_model('Run-dropout_0.5', 'Unet_dropout', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4)

In [ ]:
Unet_dropout = UNet(rate=0.3)

In [ ]:
Dropout_03 = DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_dropout)
Dropout_03.train_model('Run-dropout_0.3', 'Unet_dropout', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4)

## Weight decay
Different weight_decays are implemented for the Adam optimizer.

In [ ]:
Unet_small=UNet_small()

In [ ]:
weight_decay= DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_small)
weight_decay.train_model('Run-weightDecay_1e-4', 'Unet_small', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4, weight_decay=1e-4)

In [ ]:
weight_decay.train_model('Run-weightDecay_1e-3', 'Unet_small', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4, weight_decay=1e-3)

# Tune

## Batchsize

For tuning, the batch size is changed to 16.

In [ ]:
batchsize= DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_small)
batchsize.train_model('Run-batchsize-16', 'Unet_small', 'all', num_epochs=30, test_model=False, lr=5e-4, batch_size=16)

## Augmentation Rate

The augmentation rate is set to 0.33. The default augmentation rate of 0.66 was first run as a kind of baseline for the UNet (First run unregularized, see above) based on our Challenge X.



In [ ]:
augment= DeepModel_Trainer('../scratch/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet_small, augment_rate=0.33)
augment.train_model('Run-augmentation-0.33', 'Unet_small', 'all', num_epochs=30, test_model=False, lr=1e-3, batch_size=4)